In [ ]:
!pip install duckdb hydrotools

In [ ]:
# Import the required packages.

# Need to install TEEHR to avoid this
import sys
sys.path.insert(0, "../../src")

import holoviews as hv
import geoviews as gv
import teehr.queries.duckdb as tqd
import teehr.queries.dataframe as tqdf
import hvplot.pandas
import cartopy.crs as ccrs
from holoviews import opts

In [ ]:
# Set some notebook variables to point to the relevant study files.
PRIMARY_FILEPATH = "/home/jovyan/shared/rti-eval/post-event-example/timeseries/forcing_analysis_assim/*.parquet"
SECONDARY_FILEPATH = "/home/jovyan/shared/rti-eval/post-event-example/timeseries/forcing_medium_range/*.parquet"
CROSSWALK_FILEPATH = "/home/jovyan/shared/rti-eval/post-event-example/geo/huc10_huc10_crosswalk.parquet"
GEOMETRY_FILEPATH = "/home/jovyan/shared/rti-eval/post-event-example/geo/huc10_geometry.parquet"

In [ ]:
%%time
# Query the specified study files with a simple group_by `primary_location_id`.
# Include geometry in the response.

query_gdf = tqd.get_metrics(
        primary_filepath=PRIMARY_FILEPATH,
        secondary_filepath=SECONDARY_FILEPATH,
        crosswalk_filepath=CROSSWALK_FILEPATH,
        # geometry_filepath=GEOMETRY_FILEPATH,
        group_by=["primary_location_id", "reference_time"],
        order_by=["primary_location_id", "reference_time"],
        filters=[
            {
                "column": "primary_location_id",
                "operator": "like",
                "value": "huc10-1810020416"
            },
            {
                "column": "reference_time",
                "operator": "=",
                "value": "2022-12-18 12:00:00"
            },
        ],
        return_query=False,
        include_geometry=False,
)

In [ ]:
query_gdf 

In [ ]:
%%time
# Query the specified study files with a simple group_by `primary_location_id`.
# Include geometry in the response.

query_gdf = tqdf.get_metrics(
        primary_filepath=PRIMARY_FILEPATH,
        secondary_filepath=SECONDARY_FILEPATH,
        crosswalk_filepath=CROSSWALK_FILEPATH,
        # geometry_filepath=GEOMETRY_FILEPATH,
        group_by=["primary_location_id", "reference_time"],
        order_by=["primary_location_id"],
        filters=[
            {
                "column": "primary_location_id",
                "operator": "like",
                "value": "huc10-1810020416"
            },
            {
                "column": "reference_time",
                "operator": "=",
                "value": "2022-12-18 12:00:00"
            },
        ],
        return_query=False,
        include_geometry=False,
)

In [ ]:
query_gdf 

In [ ]:
%%time
query_gdf_prj = query_gdf.to_crs("EPSG:3857")

In [ ]:
%%time
tiles = gv.tile_sources.OSM
query_hvplot = query_gdf_prj.hvplot(
    color="red",
    crs=ccrs.GOOGLE_MERCATOR,
    hover_cols=["bias"]
)
(tiles * query_hvplot).opts(width=600, height=600, show_legend=False)

In [ ]:
%%time
query_df = tqd.get_joined_timeseries(
        primary_filepath=PRIMARY_FILEPATH,
        secondary_filepath=SECONDARY_FILEPATH,
        crosswalk_filepath=CROSSWALK_FILEPATH,
        geometry_filepath=GEOMETRY_FILEPATH,
        order_by=["primary_location_id"],
        filters=[{
            "column": "primary_location_id",
            "operator": "=",
            "value": "huc10-1801010101"

        }],
        return_query=False,
)

In [ ]:
query_df

In [ ]:
query_df.hvplot("value_time", "primary_value")